In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies.head(7640)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
7635,87785,Takers (2010),Action|Crime|Thriller
7636,87834,My Life as McDull (Mak dau goo si) (2001),Animation|Comedy|Drama
7637,87867,Zookeeper (2011),Comedy
7638,87869,Horrible Bosses (2011),Comedy|Crime


In [5]:
ratings.head(1250)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
1245,10,106696,4.5,1455306204
1246,10,106782,1.0,1455398089
1247,10,107141,2.0,1455399188
1248,10,109374,0.5,1455302315


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:
len(movies[pd.isna(movies['movieId'])==1])

0

In [8]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))
movie_genres = [change_string(g) for g in movies.genres.values]

In [9]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [10]:
genres_dict = {}
for ind in tqdm_notebook(range(0,len(movies['movieId']))):
    genres_dict[movies['movieId'][ind]] = tfidf_transformer.transform(count_vect.transform([change_string(movies.genres[ind])]))

/var/folders/ll/sf8djrcj0ljd5cltbywtgm9w0000gn/T/ipykernel_10011/3063650781.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for ind in tqdm_notebook(range(0,len(movies['movieId']))):


  0%|          | 0/9742 [00:00<?, ?it/s]

In [11]:
tags_doc = {}
for uniq in tags.movieId.unique():
    tags_doc[uniq] = ''.join(' '.join(tags[tags['movieId'] == uniq]['tag']).lower())

In [12]:
tags_count = CountVectorizer()
X_train_tags_vec = tags_count.fit_transform(tags_doc.values())
tfidf_tags = TfidfTransformer()
X_tags_train_tfidf = tfidf_tags.fit_transform(X_train_tags_vec)

In [13]:
from sklearn.neighbors import NearestNeighbors
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from scipy.spatial import distance as ds

In [14]:
neigh = NearestNeighbors(n_neighbors=1000, n_jobs=-1, metric='euclidean') 
neigh.fit(X_train_tfidf)
tag_neigh = NearestNeighbors(n_neighbors=1000, n_jobs=-1, metric='euclidean')
tag_neigh.fit(X_tags_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=1000)

In [15]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [16]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)
trainset, testset = train_test_split(data, test_size=.15)
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [17]:
def hybr_rec_sys(user,film):
    #Мой вариант заключается в том, что бы найти ближайшие фильмы по жанрам, затем из них по тегам, 
    #сделать предсказание их оценки с помощью user-based, вывести 9 лучших и 1 рандомный(размывание реккомендаций)
    
    #
    #Находим ближайшие по жанрам к примеру 1000 фильмов
    #

    near_gnrs = neigh.kneighbors(tfidf_transformer.transform(
        count_vect.transform([change_string(movies[movies['title']==film].iloc[0,2])])
    ))[1][0]
    new_near_gnrs = []
    for movid in near_gnrs:
        if (movid!=movies[movies['title']==film].iloc[0,0]):
            new_near_gnrs.append(movid)   
            
    #
    #Теперь ищем ближайших по тегам
    #
    
    if (movies[movies['title']==film].iloc[0,0] in tags_doc.keys()):
        print('TRUE')
        near_tgs = tag_neigh.kneighbors(tfidf_tags.transform(
            tags_count.transform([tags_doc.get(movies[movies['title']==film].iloc[0,0])])))[1][0]
        #оставляем 50 "лучших фильмов" в обоих метриках. Если столько не набирается, берем лучшие по жанрам, 
        #так как тегов по этому фильму может вообще не быть
        dist = []
        for gnrs in tqdm_notebook(new_near_gnrs):
            ind = 0
            while ((near_tgs[ind]!=gnrs)and(ind<999)):
                ind+=1
            if (ind != 999):
                dist.append(ind)
        if (len(dist)<50):
            for ind in range(0,1000):
                if (dist.count(ind)==0):
                    dist.append(ind)
                if (len(dist)==50):
                    break;
        else:
            dist = dist[:50]
    else: 
        dist = np.arange(50)
        
        
    marks = []
    for ind in tqdm_notebook(dist):
        marks.append(algo.predict(uid = user,iid = movies.iloc[near_gnrs[ind],1]).est)
        
    marks = [i for i in np.argsort(marks)[::-1]]
    marks = marks[:9]
    rec_ind = []
    for i in marks:
        rec_ind.append(dist[i])
    ind = np.random.choice(movies['movieId'])
    while ((rec_ind.count(ind)!=0)and(ind==movies[movies['title']==film].iloc[0,0])):
        ind = np.random.choice(movies['movieId'])
    rec_ind.append(ind)
    rec_mov = {}
    for ind in rec_ind:
        rec_mov[ind] = movies[movies['movieId']==ind][movies.columns[1:]]
    return rec_mov

In [18]:
hybr_rec_sys(1,'Kung Fu Panda 2 (2011)')

/var/folders/ll/sf8djrcj0ljd5cltbywtgm9w0000gn/T/ipykernel_10011/1905900077.py:47: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for ind in tqdm_notebook(dist):


  0%|          | 0/50 [00:00<?, ?it/s]

{15:                       title                    genres
 14  Cutthroat Island (1995)  Action|Adventure|Romance,
 22:              title                               genres
 21  Copycat (1995)  Crime|Drama|Horror|Mystery|Thriller,
 21:                 title                 genres
 20  Get Shorty (1995)  Comedy|Crime|Thriller,
 8:                  title              genres
 7  Tom and Huck (1995)  Adventure|Children,
 44:                    title                    genres
 40  Mortal Kombat (1995)  Action|Adventure|Fantasy,
 25:                        title         genres
 24  Leaving Las Vegas (1995)  Drama|Romance,
 11:                              title                genres
 10  American President, The (1995)  Comedy|Drama|Romance,
 20:                  title                              genres
 19  Money Train (1995)  Action|Comedy|Crime|Drama|Thriller,
 24:             title        genres
 23  Powder (1995)  Drama|Sci-Fi,
 47566:                     title                 genres

In [19]:
movies[movies['title']=='Kung Fu Panda 2 (2011)'].iloc[0,2]

'Action|Adventure|Animation|Children|Comedy|IMAX'